In [1]:
%pip install tianshou
%pip install pygame


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
from torch.utils.tensorboard import SummaryWriter

import tianshou as ts
from tianshou.policy import DQNPolicy

In [4]:
#start a logger
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))

#import a network that we can use
from tianshou.utils.net.common import Net
from tianshou.utils.space_info import SpaceInfo

In [3]:
#start a logger
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))

#import a network that we can use
from tianshou.utils.net.common import Net
from tianshou.utils.space_info import SpaceInfo

#get an instance of the environment
env = gym.make('CartPole-v1')

#get all the info about it
space_info = SpaceInfo.from_env(env)

#What the agent 'sees'
state_shape = space_info.observation_info.obs_shape

#what actions the agent can take
action_shape = space_info.action_info.action_shape

#build a network that takes observations and converts it to actions
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])

#this will shift the network to better predict actions/values
optim = torch.optim.Adam(net.parameters(), lr=0.001)

policy = ts.policy.DQNPolicy(
    model=net,
    optim=optim,
    discount_factor=0.9,
    action_space=env.action_space,
    estimation_step=3,
    target_update_freq=320
)

train_collector = ts.data.Collector(policy, env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True)
test_collector = ts.data.Collector(policy, env, exploration_noise=True)

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=30,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger,
).run()
print(f"Finished training in {result.timing.total_time} seconds")

/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/tianshou/data/collector.py:152: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
Epoch #1: 10001it [00:08, 1222.39it/s, env_step=10000, gradient_step=1000, len=219, n/ep=0, n/st=10, rew=219.00]                           


Epoch #1: test_reward: 165.040000 ± 63.445870, best_reward: 165.040000 ± 63.445870 in #1


Epoch #2: 10001it [00:24, 414.72it/s, env_step=20000, gradient_step=2000, len=254, n/ep=0, n/st=10, rew=254.00]                            


Epoch #2: test_reward: 365.530000 ± 106.200796, best_reward: 365.530000 ± 106.200796 in #2


Epoch #3: 10001it [00:08, 1230.29it/s, env_step=30000, gradient_step=3000, len=196, n/ep=0, n/st=10, rew=196.00]                           


...

Epoch #24: test_reward: 127.670000 ± 4.917428, best_reward: 365.530000 ± 106.200796 in #2


Epoch #25: 10001it [00:32, 308.05it/s, env_step=250000, gradient_step=25000, len=201, n/ep=0, n/st=10, rew=201.00]                           


Epoch #25: test_reward: 464.300000 ± 46.259377, best_reward: 464.300000 ± 46.259377 in #25


Epoch #26: 10001it [00:24, 402.76it/s, env_step=260000, gradient_step=26000, len=116, n/ep=0, n/st=10, rew=116.00]                           


Epoch #26: test_reward: 500.000000 ± 0.000000, best_reward: 500.000000 ± 0.000000 in #26
Finished training in 631.563648223877 seconds


In [4]:
import numpy as np

# --- Set up logging ---
logger = ts.utils.TensorboardLogger(SummaryWriter("logs/dqn"))

# --- Create the environment ---
env_name = "CartPole-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

# --- Get state and action info ---
obs_shape = train_env.observation_space.shape  
n_actions = train_env.action_space.n           
input_dim = np.prod(obs_shape)

# --- Build the DQN network ---
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, obs, state=None, info=None):
        if isinstance(obs, np.ndarray):  # Convert numpy array to tensor
            obs = torch.tensor(obs, dtype=torch.float32)
        x = torch.relu(self.fc1(obs))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x, state

net = Net(input_dim=input_dim, output_dim=n_actions)

# --- Set up optimizer ---
optim = torch.optim.Adam(net.parameters(), lr=0.001)

# --- Create the DQN policy ---
policy = ts.policy.DQNPolicy(
    model=net,
    optim=optim,
    discount_factor=0.9,
    action_space=train_env.action_space,
    estimation_step=3,
    target_update_freq=320
)

# --- Set up data collectors ---
train_collector = ts.data.Collector(
    policy, 
    train_env, 
    ts.data.VectorReplayBuffer(20000, 1),
    exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# --- Train the agent ---
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=30,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=logger,
).run()

print(f"Finished training in {result.timing.total_time} seconds")
torch.save(policy.state_dict(), 'models/dqn.pth')

# --- Evaluate the trained agent ---
policy.load_state_dict(torch.load('models/dqn.pth'))
policy.eval()
policy.set_eps(0.05)

# Create an environment with rendering enabled.
eval_env = gym.make(env_name, render_mode="human")
eval_env.reset()
collector = ts.data.Collector(policy, eval_env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35, reset_before_collect=True)
eval_env.close()

Epoch #1: 10001it [00:07, 1380.31it/s, env_step=10000, gradient_step=1000, len=97, n/ep=0, n/st=10, rew=97.00]                           


Epoch #1: test_reward: 101.150000 ± 13.622316, best_reward: 101.150000 ± 13.622316 in #1


Epoch #2: 10001it [00:07, 1392.42it/s, env_step=20000, gradient_step=2000, len=166, n/ep=0, n/st=10, rew=166.00]                           


Epoch #2: test_reward: 188.260000 ± 54.130328, best_reward: 188.260000 ± 54.130328 in #2


Epoch #3: 10001it [00:07, 1417.54it/s, env_step=30000, gradient_step=3000, len=205, n/ep=0, n/st=10, rew=205.00]                           


...

Epoch #8: 10001it [00:07, 1373.51it/s, env_step=80000, gradient_step=8000, len=189, n/ep=0, n/st=10, rew=189.00]                           


Epoch #8: test_reward: 190.190000 ± 10.045591, best_reward: 279.570000 ± 60.867110 in #5


Epoch #9: 10001it [00:06, 1475.09it/s, env_step=90000, gradient_step=9000, len=219, n/ep=0, n/st=10, rew=219.00]                           


Epoch #9: test_reward: 499.580000 ± 4.178947, best_reward: 499.580000 ± 4.178947 in #9
Finished training in 112.15051007270813 seconds


/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/tianshou/data/collector.py:152: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [5]:
import numpy as np

class DynamicCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, gravity_range=(5.0, 25.0), length_range=(0.2, 1.0), mass_range=(0.05, 1.0), friction_range=(0.0, 0.5)):
        super().__init__(env)
        self.gravity_range = gravity_range
        self.length_range = length_range
        self.mass_range = mass_range
        self.friction_range = friction_range
        self.episode_count = 0  # Track episodes

    def reset(self, **kwargs):
        self.episode_count += 1  # Increment episode count

        # Gradually shift physics conditions over time
        new_gravity = np.interp(self.episode_count, [0, 500], self.gravity_range)  # Gradually change gravity
        new_length = np.interp(self.episode_count, [0, 500], self.length_range)  # Change pole length
        new_mass = np.interp(self.episode_count, [0, 500], self.mass_range)  # Change pole mass
        new_friction = np.interp(self.episode_count, [0, 500], self.friction_range)  # Change friction

        # Apply new parameters
        self.env.env.gravity = new_gravity
        self.env.env.length = new_length
        self.env.env.masspole = new_mass
        self.env.env.force_mag = 10.0 * (new_mass / 0.1)  # Scale force with mass to balance difficulty
        self.env.env.friction = new_friction

        print(f"Episode {self.episode_count}: Gravity={new_gravity}, Length={new_length}, Mass={new_mass}, Friction={new_friction}")

        return self.env.reset(**kwargs)

env_name = "CartPole-v1"
base_env = gym.make(env_name)
train_env = DynamicCartPoleWrapper(base_env)

test_env = DynamicCartPoleWrapper(gym.make(env_name))  # Use same variations for testing

train_collector = ts.data.Collector(
    policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# Load the pre-trained model from Code Block 1 
policy.load_state_dict(torch.load('models/dqn.pth')) 
policy.eval()

best_mean_reward = -np.inf 

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/dqn_v1_alternate_metrics.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

# Train the model while CBP adapts to the changing environment
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=30,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/dynamic_cartpole")),
).run()

save_best_model(result.best_reward)

Episode 1: Gravity=5.04, Length=0.2016, Mass=0.0519, Friction=0.001
Episode 1: Gravity=5.04, Length=0.2016, Mass=0.0519, Friction=0.001
Episode 2: Gravity=5.08, Length=0.20320000000000002, Mass=0.0538, Friction=0.002
Episode 3: Gravity=5.12, Length=0.2048, Mass=0.0557, Friction=0.003
Episode 4: Gravity=5.16, Length=0.2064, Mass=0.057600000000000005, Friction=0.004
Episode 5: Gravity=5.2, Length=0.20800000000000002, Mass=0.059500000000000004, Friction=0.005
Episode 6: Gravity=5.24, Length=0.2096, Mass=0.0614, Friction=0.006
Episode 7: Gravity=5.28, Length=0.2112, Mass=0.0633, Friction=0.007
Episode 8: Gravity=5.32, Length=0.21280000000000002, Mass=0.06520000000000001, Friction=0.008
Episode 9: Gravity=5.36, Length=0.2144, Mass=0.0671, Friction=0.009000000000000001
Episode 10: Gravity=5.4, Length=0.21600000000000003, Mass=0.069, Friction=0.01
Episode 11: Gravity=5.44, Length=0.21760000000000002, Mass=0.0709, Friction=0.011
Episode 12: Gravity=5.48, Length=0.2192, Mass=0.0728, Friction=0.

Epoch #1:   5%|4         | 490/10000 [00:00<00:05, 1801.02it/s, env_step=490, gradient_step=49, len=0, n/ep=0, n/st=10, rew=0.00]

Episode 3: Gravity=5.12, Length=0.2048, Mass=0.0557, Friction=0.003
Episode 207: Gravity=13.28, Length=0.5312, Mass=0.44329999999999997, Friction=0.20700000000000002
Episode 208: Gravity=13.32, Length=0.5328, Mass=0.4452, Friction=0.20800000000000002
Episode 209: Gravity=13.36, Length=0.5344, Mass=0.4471, Friction=0.209
Episode 210: Gravity=13.4, Length=0.536, Mass=0.449, Friction=0.21
Episode 211: Gravity=13.44, Length=0.5376000000000001, Mass=0.45089999999999997, Friction=0.211
Episode 212: Gravity=13.48, Length=0.5392, Mass=0.4528, Friction=0.212
Episode 213: Gravity=13.52, Length=0.5408, Mass=0.4547, Friction=0.213
Episode 214: Gravity=13.56, Length=0.5424, Mass=0.4566, Friction=0.214
Episode 215: Gravity=13.6, Length=0.544, Mass=0.45849999999999996, Friction=0.215
Episode 216: Gravity=13.64, Length=0.5456000000000001, Mass=0.4604, Friction=0.216
Episode 217: Gravity=13.68, Length=0.5472, Mass=0.4623, Friction=0.217
Episode 218: Gravity=13.72, Length=0.5488, Mass=0.4642, Friction=0

Epoch #1:   8%|8         | 830/10000 [00:10<02:18, 66.16it/s, env_step=830, gradient_step=83, len=264, n/ep=0, n/st=10, rew=264.00]  

Episode 307: Gravity=17.28, Length=0.6912, Mass=0.6333000000000001, Friction=0.307
Episode 308: Gravity=17.32, Length=0.6928000000000001, Mass=0.6352000000000001, Friction=0.308
Episode 4: Gravity=5.16, Length=0.2064, Mass=0.057600000000000005, Friction=0.004


...

Episode 48: Gravity=6.92, Length=0.27680000000000005, Mass=0.1412, Friction=0.048
Episode 49: Gravity=6.96, Length=0.2784, Mass=0.1431, Friction=0.049


Epoch #1:  96%|#########6| 9600/10000 [00:16<00:00, 1496.30it/s, env_step=9600, gradient_step=960, len=160, n/ep=0, n/st=10, rew=160.00]

Episode 50: Gravity=7.0, Length=0.28, Mass=0.14500000000000002, Friction=0.05
Episode 51: Gravity=7.04, Length=0.2816, Mass=0.1469, Friction=0.051000000000000004
Episode 52: Gravity=7.08, Length=0.2832, Mass=0.1488, Friction=0.052000000000000005


Epoch #1: 10001it [00:16, 608.54it/s, env_step=10000, gradient_step=1000, len=198, n/ep=0, n/st=10, rew=198.00]                            


Episode 53: Gravity=7.12, Length=0.2848, Mass=0.1507, Friction=0.053
Episode 54: Gravity=7.16, Length=0.2864, Mass=0.1526, Friction=0.054
Episode 309: Gravity=17.36, Length=0.6944, Mass=0.6371, Friction=0.309
Episode 310: Gravity=17.4, Length=0.696, Mass=0.639, Friction=0.31
Episode 311: Gravity=17.439999999999998, Length=0.6976, Mass=0.6409, Friction=0.311
Episode 312: Gravity=17.48, Length=0.6992, Mass=0.6428, Friction=0.312
Episode 313: Gravity=17.52, Length=0.7008000000000001, Mass=0.6447, Friction=0.313
Episode 314: Gravity=17.560000000000002, Length=0.7024000000000001, Mass=0.6466000000000001, Friction=0.314
Episode 315: Gravity=17.6, Length=0.704, Mass=0.6485000000000001, Friction=0.315
Episode 316: Gravity=17.64, Length=0.7056, Mass=0.6504000000000001, Friction=0.316
Episode 317: Gravity=17.68, Length=0.7072, Mass=0.6523, Friction=0.317
Episode 318: Gravity=17.72, Length=0.7088000000000001, Mass=0.6542, Friction=0.318
Episode 319: Gravity=17.759999999999998, Length=0.7104000000

In [6]:
import numpy as np

class NoisyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, noise_std=0.1):
        super().__init__(env)
        self.noise_std = noise_std  # Standard deviation of noise

    def step(self, action):
        if np.random.rand() < self.noise_std:  # With probability `noise_std`, flip action
            action = 1 - action
        return self.env.step(int(action))


class WindyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, wind_max=5.0, wind_change_prob=0.1):
        super().__init__(env)
        self.wind_max = wind_max  # Max lateral force
        self.wind_force = 0
        self.wind_change_prob = wind_change_prob  # Probability of wind change per step

    def step(self, action):
        # Occasionally change wind force
        if np.random.rand() < self.wind_change_prob:
            self.wind_force = np.random.uniform(-self.wind_max, self.wind_max)

        # Get the step results
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Apply wind effect by modifying the observation directly
        # CartPole observation: [position, velocity, pole_angle, pole_angular_velocity]
        obs[1] += self.wind_force * 0.01  # Modify cart velocity in the observation
        
        # Update the environment's state array
        self.env.unwrapped.state = obs.copy()

        return obs, reward, terminated, truncated, info

from collections import deque

class DelayedActionCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, delay_steps=3):
        super().__init__(env)
        self.delay_steps = delay_steps  # How many steps before action takes effect
        # self.action_queue = deque([0] * delay_steps, maxlen=delay_steps)  # Store past actions
        self.action_queue = deque(np.random.choice([0, 1], delay_steps), maxlen=delay_steps)

    def step(self, action):
        self.action_queue.append(action)  # Queue the new action
        delayed_action = self.action_queue.popleft()  # Use delayed action
        return self.env.step(delayed_action)  # Will automatically pass through all 5 values

train_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))
test_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))

# policy.load_state_dict(torch.load('models/cbp_dqn_adapted_v1.pth')) # Should change name to cbp_dqn_adapted_v1.pth, but haven't
policy.load_state_dict(torch.load('models/dqn_v1_alternate_metrics.pth'))
policy.train()
policy.set_eps(0.1)  # Start with exploration

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/dqn_final_adapted_v2.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=ts.data.Collector(policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(policy, test_env, exploration_noise=True),
    max_epoch=30,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/windy_noisy_delayed")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:03, 1374.86it/s, env_step=5000, gradient_step=500, len=43, n/ep=0, n/st=10, rew=43.00]                            


Epoch #1: test_reward: 123.050000 ± 69.809365, best_reward: 123.050000 ± 69.809365 in #1


Epoch #2: 5001it [00:03, 1388.93it/s, env_step=10000, gradient_step=1000, len=30, n/ep=0, n/st=10, rew=30.00]                          


Epoch #2: test_reward: 99.850000 ± 49.130718, best_reward: 123.050000 ± 69.809365 in #1


Epoch #3: 5001it [00:03, 1372.12it/s, env_step=15000, gradient_step=1500, len=102, n/ep=0, n/st=10, rew=102.00]                          


...

Epoch #29: 5001it [00:03, 1421.58it/s, env_step=145000, gradient_step=14500, len=158, n/ep=0, n/st=10, rew=158.00]                          


Epoch #29: test_reward: 172.850000 ± 75.132733, best_reward: 190.950000 ± 116.081642 in #23


Epoch #30: 5001it [00:03, 1516.18it/s, env_step=150000, gradient_step=15000, len=164, n/ep=1, n/st=10, rew=164.00]                          


Epoch #30: test_reward: 126.150000 ± 66.820861, best_reward: 190.950000 ± 116.081642 in #23
Finished training in 126.92998313903809 seconds
New best model saved with mean reward: 190.95


: 

In [7]:
# First create a new policy with correct dimensions for Acrobot
env_name = "Acrobot-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

input_dim = train_env.observation_space.shape[0]  # Should be 6
output_dim = train_env.action_space.n  # Should be 3

# Create a new model with correct dimensions
# new_net = Net(input_dim=input_dim, output_dim=output_dim)
new_net = Net(state_shape=input_dim, action_shape=output_dim, hidden_sizes=[128, 128, 128])


# Create a new policy
new_policy = DQNPolicy(
    model=new_net,
    optim=torch.optim.Adam(new_net.parameters(), lr=0.001),
    discount_factor=0.975,
    estimation_step=3,
    target_update_freq=1000,
    action_space=train_env.action_space
)

# Load the pretrained weights for matching layers
pretrained_dict = torch.load("models/dqn_final_adapted_v2.pth")
model_dict = new_policy.state_dict()

# Filter out incompatible layers
filtered_dict = {k: v for k, v in pretrained_dict.items() 
                if k in model_dict and 
                'hidden_layers.0' not in k and  # Skip first layer
                'out_layer' not in k and        # Skip output layer
                model_dict[k].shape == v.shape}

# Update matching layers
model_dict.update(filtered_dict)
new_policy.load_state_dict(model_dict)

# Now you can train with the new policy
new_policy.train()
new_policy.set_eps(0.1)

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(new_policy.state_dict(), 'models/dqn_Acrobot-v1.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=new_policy,
    train_collector=ts.data.Collector(new_policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(new_policy, test_env, exploration_noise=True),
    max_epoch=20,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: new_policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: new_policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/Acrobot-v1")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/tianshou/data/collector.py:152: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
Epoch #1: 5001it [00:07, 707.49it/s, env_step=5000, gradient_step=500, len=430, n/ep=0, n/st=10, rew=-429.00]                          


Epoch #1: test_reward: -276.650000 ± 93.203152, best_reward: -219.450000 ± 63.378604 in #0


Epoch #2: 5001it [00:06, 807.89it/s, env_step=10000, gradient_step=1000, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #2: test_reward: -500.000000 ± 0.000000, best_reward: -219.450000 ± 63.378604 in #0


Epoch #3: 5001it [00:05, 983.16it/s, env_step=15000, gradient_step=1500, len=500, n/ep=0, n/st=10, rew=-500.00]                           


...

Epoch #18: test_reward: -491.300000 ± 31.900000, best_reward: -140.250000 ± 47.803635 in #12


Epoch #19: 5001it [00:06, 730.81it/s, env_step=95000, gradient_step=9500, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #19: test_reward: -500.000000 ± 0.000000, best_reward: -140.250000 ± 47.803635 in #12


Epoch #20: 5001it [00:07, 691.73it/s, env_step=100000, gradient_step=10000, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #20: test_reward: -500.000000 ± 0.000000, best_reward: -140.250000 ± 47.803635 in #12
Finished training in 202.31195306777954 seconds
New best model saved with mean reward: -140.25


In [11]:
# First create a new policy with correct dimensions for Acrobot
env_name = "MountainCar-v0"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

input_dim = train_env.observation_space.shape[0]  # Should be 6
output_dim = train_env.action_space.n  # Should be 3

# Create a new model with correct dimensions
# new_net = Net(input_dim=input_dim, output_dim=output_dim)
new_net = Net(state_shape=input_dim, action_shape=output_dim, hidden_sizes=[128, 128, 128])


# Create a new policy
new_policy = DQNPolicy(
    model=new_net,
    optim=torch.optim.Adam(new_net.parameters(), lr=0.001),
    discount_factor=0.975,
    estimation_step=3,
    target_update_freq=1000,
    action_space=train_env.action_space
)

# Load the pretrained weights for matching layers
pretrained_dict = torch.load("models/dqn_final_adapted_v2.pth")
model_dict = new_policy.state_dict()

# Filter out incompatible layers
filtered_dict = {k: v for k, v in pretrained_dict.items() 
                if k in model_dict and 
                'hidden_layers.0' not in k and  # Skip first layer
                'out_layer' not in k and        # Skip output layer
                model_dict[k].shape == v.shape}

# Update matching layers
model_dict.update(filtered_dict)
new_policy.load_state_dict(model_dict)

# Now you can train with the new policy
new_policy.train()
new_policy.set_eps(0.1)

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(new_policy.state_dict(), 'models/dqn_MountainCar-v0.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=new_policy,
    train_collector=ts.data.Collector(new_policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(new_policy, test_env, exploration_noise=True),
    max_epoch=20,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: new_policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: new_policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/MountainCar-v0")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/tianshou/data/collector.py:152: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
Epoch #1: 5001it [00:07, 657.80it/s, env_step=5000, gradient_step=500, len=200, n/ep=1, n/st=10, rew=-200.00]                          


Epoch #1: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #2: 5001it [00:07, 694.38it/s, env_step=10000, gradient_step=1000, len=200, n/ep=1, n/st=10, rew=-200.00]                          


Epoch #2: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #3: 5001it [00:06, 728.18it/s, env_step=15000, gradient_step=1500, len=200, n/ep=1, n/st=10, rew=-200.00]                          


...

Epoch #18: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #19: 5001it [00:06, 776.12it/s, env_step=95000, gradient_step=9500, len=200, n/ep=0, n/st=10, rew=-200.00]                          


Epoch #19: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #20: 5001it [00:06, 769.04it/s, env_step=100000, gradient_step=10000, len=200, n/ep=1, n/st=10, rew=-200.00]                          


Epoch #20: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0
Finished training in 164.70815896987915 seconds
New best model saved with mean reward: -200.0
